<h1>Trabajdo Realizado para la Feria</h1>

<h2>Integrantes</h2>
<p>Luis Caballero 8-9231440<br>
Ángel Cilli<br>
Jaime Acosta</p>

<p><b>Idea o Concepto:</b>Trabajaremos un reconocedor de "personal" dentro del equipo, un Aruco distinto para Jaime Acosta y Luis Caballero, que cuando sean mostrados a la cámara el programa ejecutará distintos enfoques con Aruco.</p>

<h2>Detector del Personal</h2>

<p>Para el detector de personal usaremos el diccionario <b>DISC_4x4_1000</b>b><br>
Marcado ID 15 → Luis Caballero<br>
Marcador ID 13 → Jaime Acosta</p>

In [3]:
#IMPORTACIONES
import numpy as np
import cv2
from matplotlib import pyplot as plt
import os
import pickle

#AruCO

OVERLAY_SIZE_PER = 1

# Check for camera calibration data
if not os.path.exists('calibration.pckl'):
    print("Necesitas Calibrar tu Cámara para un buen uso")
    exit()
else:
    f = open('calibration.pckl', 'rb')
    cameraMatrix, distCoeffs = pickle.load(f)
    f.close()
    if cameraMatrix is None or distCoeffs is None:
        print("Alguien salio mal. Recalibra tu cámara")
        exit()


# Create the ArUco dictionary and parameters:
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_1000) #Detector de Personal
aruco_dict2 = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_5X5_1000) #Imagen 3D
aruco_dict3 = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_1000) #Filtros
parameters = cv2.aruco.DetectorParameters()

# Create the video capture object:
video_capture = cv2.VideoCapture(0)
frame_index = 0;

while video_capture.isOpened():
    ret, frame = video_capture.read()
    if not ret:
        break

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    corners, ids, rejectedImgPoints = cv2.aruco.detectMarkers(gray_frame, aruco_dict, parameters=parameters)

    if ids is not None:
        for i in range(len(ids)):
            if ids[i] == 15:
                # Mensaje de bienvenida para Luis Caballero
                cv2.putText(img=frame, text="Bienvenido Luis", org=(10, 30),
            fontFace=cv2.FONT_HERSHEY_COMPLEX, fontScale=1,
            color=(102, 102, 255), thickness=2)
                #aqui deberia ir lo de filtros o video
            
            elif ids[i] == 13:
                # Mensaje de bienvenida para Jaime Acosta
                cv2.putText(img=frame, text="Bienvenido Jaime", org=(10, 30),
            fontFace=cv2.FONT_HERSHEY_COMPLEX, fontScale=1,
            color=(102, 102, 255), thickness=2)
                #aqui deberia ir lo del 3D
                
            else:
                #Mensaje de no esta registrado
                cv2.putText(img=frame, text="Quien tu eres!?", org=(10, 30),
            fontFace=cv2.FONT_HERSHEY_COMPLEX, fontScale=1,
            color=(102, 102, 255), thickness=2)
                

    cv2.aruco.drawDetectedMarkers(frame, corners, ids)
    cv2.imshow('ArUco Marker and Face Detection', frame)
    

    if cv2.waitKey(20) & 0xFF == ord('c'):
        frame_name = "out/Parcial_frame_{}.png".format(frame_index)
        cv2.imwrite(frame_name, frame)
        frame_index += 1

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()


<hr>

In [1]:
# Import required packages
import cv2
import os
import pickle
import numpy as np

OVERLAY_SIZE_PER = 1

# Check for camera calibration data
if not os.path.exists('calibration.pckl'):
    print("You need to calibrate the camera before")
    exit()
else:
    f = open('calibration.pckl', 'rb')
    cameraMatrix, distCoeffs = pickle.load(f)
    f.close()
    if cameraMatrix is None or distCoeffs is None:
        print("Something went wrong. Recalibrate the camera")
        exit()

# Load the video overlay:
video_path = "Img/SheidexPapure.mp4"  # Cambia a la ruta de tu video
cap_overlay = cv2.VideoCapture(video_path)

# Function to get the next frame from the overlay video
def get_next_overlay_frame():
    ret, frame = cap_overlay.read()
    if not ret:
        cap_overlay.set(cv2.CAP_PROP_POS_FRAMES, 0)
        ret, frame = cap_overlay.read()
    return frame

def draw_augmented_overlay(pts_1, overlay_image, image):
    """Overlay the image 'overlay_image' onto the image 'image'"""

    # Define the squares of the overlay_image image to be drawn:
    pts_2 = np.float32([[0, 0], [overlay_image.shape[1], 0], [overlay_image.shape[1], overlay_image.shape[0]],
                        [0, overlay_image.shape[0]]])

    # Draw border to see the limits of the image:
    cv2.rectangle(overlay_image, (0, 0), (overlay_image.shape[1], overlay_image.shape[0]), (255, 255, 0), 10)

    # Create the transformation matrix:
    M = cv2.getPerspectiveTransform(pts_2, pts_1)

    # Transform the overlay_image image using the transformation matrix M:
    dst_image = cv2.warpPerspective(overlay_image, M, (image.shape[1], image.shape[0]))

    # Create the mask:
    dst_image_gray = cv2.cvtColor(dst_image, cv2.COLOR_BGR2GRAY)
    ret, mask = cv2.threshold(dst_image_gray, 0, 255, cv2.THRESH_BINARY_INV)

    # Compute bitwise conjunction using the calculated mask:
    image_masked = cv2.bitwise_and(image, image, mask=mask)

    # Add the two images to create the resulting image:
    result = cv2.add(dst_image, image_masked)
    return result


# Create the dictionary object and the parameters:
dictionary = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_7X7_250)
parameters = cv2.aruco.DetectorParameters()
detector = cv2.aruco.ArucoDetector(dictionary, parameters)

# Create video capture object 'capture' to be used to capture frames from the first connected camera:
capture = cv2.VideoCapture(0)

while True:
    # Capture frame by frame from the video capture object 'capture':
    ret, frame = capture.read()

    # We convert the frame to grayscale:
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect markers:
    corners, ids, rejectedImgPoints = detector.detectMarkers(gray_frame)

    # Draw detected markers:
    frame = cv2.aruco.drawDetectedMarkers(image=frame, corners=corners, ids=ids, borderColor=(0, 255, 0))

    if ids is not None:
        # rvecs and tvecs are the rotation and translation vectors respectively, for each of the markers in corners.
        rvecs, tvecs, _ = cv2.aruco.estimatePoseSingleMarkers(corners, 1, cameraMatrix, distCoeffs)

        for rvec, tvec in zip(rvecs, tvecs):
            # Note: The marker coordinate system is centered on the center of the marker
            # The coordinates of the four corners of the marker (in its own coordinate system) are:
            # 1: (-markerLength/2, markerLength/2, 0)
            # 2: (markerLength/2, markerLength/2, 0)
            # 3: (markerLength/2, -markerLength/2, 0)
            # 4: (-markerLength/2, -markerLength/2, 0)
            # Define the points where you want the image to be overlaid (remember: marker coordinate system):
            desired_points = np.float32(
                [[-1 / 2, 1 / 2, 0], [1 / 2, 1 / 2, 0], [1 / 2, -1 / 2, 0], [-1 / 2, -1 / 2, 0]]) * OVERLAY_SIZE_PER

            # Project the points:
            projected_desired_points, jac = cv2.projectPoints(desired_points, rvecs, tvecs, cameraMatrix, distCoeffs)

            # Get the next frame from the overlay video
            overlay_frame = get_next_overlay_frame()

            # Overlay the video frame onto the camera frame
            frame = draw_augmented_overlay(projected_desired_points, overlay_frame, frame)

    # Display the resulting augmented frame:
    cv2.imshow('frame', frame)

    # Press 'q' to exit:
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release everything:
capture.release()
cap_overlay.release()
cv2.destroyAllWindows()


In [26]:
import cv2
import numpy as np

# Configuración del sistema de marcadores ArUco
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
aruco_params = cv2.aruco.DetectorParameters()

# IDs permitidos para el acceso y sus respectivos nombres
allowed_marker_ids = {13: "Jaime Acosta", 15: "Luis Caballero"}

# Función para verificar si el marcador permitido está presente y dar la bienvenida
def check_access(frame):
    # Convertir a escala de grises
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detectar marcadores ArUco
    corners, ids, rejected = cv2.aruco.detectMarkers(gray, aruco_dict, parameters=aruco_params)

    # Si se detectan marcadores
    if ids is not None:
        for marker_id in ids:
            if marker_id[0] in allowed_marker_ids:
                # Obtener el nombre asociado al ID del marcador
                name = allowed_marker_ids[marker_id[0]]
                if marker_id[0] == 15:
                    text_color = (0, 0, 0)
                    bienvenida(frame, name, marker_id[0], text_color)
                    
                if marker_id[0] == 13:
                    text_color = (255, 0, 0)
                    bienvenida(frame, name, marker_id[0], text_color)       
                
        return corners, ids

    # Imprimir mensaje de acceso denegado si no se detecta ningún marcador permitido
    text_color =(0, 255, 0)
    denegado(frame, "Desconocido", -1, text_color)
    return None, None

# Configuración de la cámara
cap = cv2.VideoCapture(0)

while True:
    # Capturar un frame
    ret, frame = cap.read()

    # Verificar acceso basado en el marcador ArUco
    corners, ids = check_access(frame)

    # Mostrar el frame con el marcador dibujado
    if corners is not None:
        cv2.aruco.drawDetectedMarkers(frame, corners, ids)
    cv2.imshow('Acceso basado en marcador ArUco', frame)

    # Salir del bucle si se presiona 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar recursos
cap.release()
cv2.destroyAllWindows()

def bienvenida(frame, name, marker_id, text_color):
    cv2.putText(img=frame, text=f"Bienvenido, {name} (ID {marker_id}). Acceso permitido.",
                org=(10, 30), fontFace=cv2.FONT_HERSHEY_COMPLEX, fontScale=0.6,
                color=text_color, thickness=2)
    
def denegado(frame, name, marker_id, text_color):
    cv2.putText(img=frame, text=f"{name} (ID {marker_id}). Quien eres?",
                org=(10, 30), fontFace=cv2.FONT_HERSHEY_COMPLEX, fontScale=0.6,
                color=text_color, thickness=2)




error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [3]:
import cv2
from cv2 import aruco

# Inicializar el diccionario 4x4
dictionary_4x4 = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)

# Inicializar el diccionario 5x5
dictionary_5x5 = aruco.getPredefinedDictionary(aruco.DICT_5X5_50)

# Inicializar el diccionario 6x6
dictionary_6x6 = aruco.getPredefinedDictionary(aruco.DICT_6X6_250)

# Cargar el clasificador Haar Cascade para la detección facial
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Inicializar la cámara (asegúrate de tener una cámara disponible)
cap = cv2.VideoCapture(0)

while True:
    # Capturar un fotograma de la cámara
    ret, frame = cap.read()

    # Convertir el fotograma a escala de grises
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detectar marcadores ArUco en la imagen
    corners, ids, _ = aruco.detectMarkers(gray, dictionary_4x4)

    if ids is not None and len(ids) > 0:
        # Si se detecta un marcador con ID 15 (Luis Caballero)
        if 15 in ids:
            # Cambiar al diccionario 5x5
            dictionary = dictionary_5x5

            # Mostrar video si se detecta el marcador con ID 20
            if 20 in ids:
                # Coloca aquí el código para mostrar el video
                pass

        # Si se detecta un marcador con ID 13 (Jaime Acosta)
        elif 13 in ids:
            # Cambiar al diccionario 6x6
            dictionary = dictionary_6x6

            # Si se detecta el marcador con ID 10, aplicar el filtro de blur en la cara
            if 10 in ids:
                # Detectar caras en la imagen
                faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

                # Aplicar blur a las caras detectadas
                for (x, y, w, h) in faces:
                    face_roi = frame[y:y + h, x:x + w]
                    face_roi = cv2.GaussianBlur(face_roi, (99, 99), 30)

                    # Reemplazar la región de la cara con blur en el fotograma original
                    frame[y:y + face_roi.shape[0], x:x + face_roi.shape[1]] = face_roi

        # Dibujar los marcadores detectados en la imagen original
        aruco.drawDetectedMarkers(frame, corners, ids)

    # Mostrar la imagen resultante
    cv2.imshow('ARUCO Access Control', frame)

    # Romper el bucle si se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar los recursos
cap.release()
cv2.destroyAllWindows()
